In [1]:
import cv2
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display

def inverse_image(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    norm = img / 255.0
    corrected = np.power(norm, gamma)
    return np.uint8(corrected * 255)

def log_transform(img):
    c = 255 / np.log1p(np.max(img))
    return np.uint8(c * np.log1p(img))

def histogram_equalization(img):
    return cv2.equalizeHist(img)

def contrast_stretching(img):
    min_val = np.min(img)
    max_val = np.max(img)
    stretched = (img - min_val) * 255.0 / (max_val - min_val)
    return np.uint8(stretched)

transform_funcs = {
    'Image Inverse (I)': ('inverse', inverse_image),
    'Gamma Correction (G)': ('gamma', gamma_correction),
    'Log Transformation (L)': ('log', log_transform),
    'Histogram Equalization (H)': ('hist_eq', histogram_equalization),
    'Contrast Stretching (C)': ('contrast', contrast_stretching)
}

# Dropdown chọn phương pháp
dropdown = widgets.Dropdown(
    options=list(transform_funcs.keys()),
    description='Biến đổi:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Nút chạy
button = widgets.Button(description='Chạy biến đổi', button_style='success')

# Xử lý khi nhấn nút
def on_button_click(b):
    choice = dropdown.value
    name, func = transform_funcs[choice]

    for file in os.listdir("exercise"):
        path = os.path.join("exercise", file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Bỏ qua {file}")
            continue

        out = func(img)
        save_path = os.path.join("exercise", f"{name}_{file}")
        cv2.imwrite(save_path, out)
        cv2.imshow(f"{name}_{file}", out)
        key = cv2.waitKey(0)
        if key == 27:
            break
    cv2.destroyAllWindows()

button.on_click(on_button_click)

# Hiển thị giao diện
display(dropdown, button)


Dropdown(description='Biến đổi:', layout=Layout(width='50%'), options=('Image Inverse (I)', 'Gamma Correction …

Button(button_style='success', description='Chạy biến đổi', style=ButtonStyle())

In [1]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import ttk, messagebox

def fft_image(img):
    dft = np.fft.fft2(img)
    dft_shift = np.fft.fftshift(dft)
    magnitude = 20 * np.log1p(np.abs(dft_shift)) 
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(magnitude)

def butterworth_lowpass(img, D0=30, n=2):
    rows, cols = img.shape
    V, U = np.meshgrid(np.arange(cols) - cols // 2, np.arange(rows) - rows // 2)
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (D / D0)**(2 * n))
    F = np.fft.fftshift(np.fft.fft2(img))
    G = H * F
    out = np.fft.ifft2(np.fft.ifftshift(G))
    out = np.abs(out)
    out = cv2.normalize(out, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(out)

def butterworth_highpass(img, D0=30, n=2):
    low = butterworth_lowpass(img, D0, n)
    high = img.astype(np.float32) - low.astype(np.float32)
    high = cv2.normalize(high, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(high)

transform_fft = {
    'Fast Fourier Transform': ('fft', fft_image),
    'Butterworth Lowpass Filter': ('butter_lpf', butterworth_lowpass),
    'Butterworth Highpass Filter': ('butter_hpf', butterworth_highpass)
}

def apply_filter(method_name):
    if method_name not in transform_fft:
        messagebox.showerror("Lỗi", "Phương pháp không hợp lệ!")
        return

    name, func = transform_fft[method_name]

    for file in os.listdir("exercise"):
        path = os.path.join("exercise", file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Bỏ qua {file} (không phải ảnh).")
            continue

        out = func(img)
        save_path = os.path.join("exercise", f"{name}_{file}")
        cv2.imwrite(save_path, out)

        cv2.imshow(f"{name}_{file}", out)
        if cv2.waitKey(0) == 27:  
            break

    cv2.destroyAllWindows()

def create_ui():
    root = tk.Tk()
    root.title("Bài 2 - Biến đổi ảnh Fourier và Butterworth")

    tk.Label(root, text="Chọn phương pháp biến đổi ảnh:", font=("Arial", 12)).pack(pady=10)

    selected_method = tk.StringVar()
    dropdown = ttk.Combobox(root, textvariable=selected_method, width=35, state="readonly")
    dropdown['values'] = list(transform_fft.keys())
    dropdown.current(0)
    dropdown.pack(pady=10)

    apply_btn = tk.Button(root, text="Thực hiện biến đổi", font=("Arial", 11), command=lambda: apply_filter(selected_method.get()))
    apply_btn.pack(pady=20)

    root.mainloop()

create_ui()


In [3]:
import random

def swap_rgb(img):
    return img[:, :, np.random.permutation(3)]

def random_transform1(img):
    _, func = random.choice(list(transform_funcs.values()))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return func(gray)

def run_random_rgb_transform():
    for file in os.listdir("exercise"):
        img = cv2.imread(f"exercise/{file}")
        if img is None: continue
        swapped = swap_rgb(img)
        transformed = random_transform1(swapped)
        cv2.imwrite(f"exercise/random1_{file}", transformed)
        cv2.imshow("random1", transformed)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

run_random_rgb_transform()


: 